In [1]:
import pandas as pd
import numpy as np

# QUESTION 3
Finally, modify your code accordingly in order to price an exotic option with European payoff. In this case we are going to price an Up-and-Out (UAO) European Call Option. The payoff at maturity (T) of such option is as follows:

$v(S_T)=(𝑆_T−𝐾)^+$ given $max_{[0,𝑇]}𝑆_t<𝐿$

where L is the barrier level such that, if trespassed, the value of the option becomes 0. 

Notice that the value of this option is path-dependent, not just on terminal value. The maturity of the option is 1 year, and is currently struck at-the-money (ATM).

The up-and-out barrier for the option is USD 130, while the current underlying stock price is USD  100. Consider the Binomial model under the same u, d, and r parameters from part (1) in this submission and 5 steps in the pricing process.

In [2]:
S2 = 100
L = 130
X2 = 100
# Group Number
group_number = 2
u = 1.10 + group_number/100; d = 1/u

In [3]:
def recursive_name(u,d,n):
    if n <=1:
        return [u,d]
    else:
        te = recursive_name(u,d,n-1)
        total_list = []
        for i2 in te:
            for i1 in [u,d]:
                total_list.append(i2+i1)
        return total_list

In [4]:
def cal_exotic_option(s,x,l,u,d,n,t,r):
    p = (1-d)/(u-d)
    p_inv = 1-p
    temp = np.ones((2**n,n+2))*s
    price_df = pd.DataFrame(temp, columns=['w']+[f'P({i})' for i in range(n+1)])
    price_df['w'] = recursive_name('u','d',n)
    dic = {}
    def cal_price_prob(a1,i1,u1,d1,s1,t1):
        te_str = a1['w'][:i1]
        u_count = te_str.count('u')
        d_count = te_str.count('d')
        if t1=='price':
            return s1*(u1**u_count)*(d1**d_count)
        elif t1=='prob':
            return (p**u_count)*(p_inv**d_count)
    for i in range(n+1):
        price_df[f'P({i})'] = price_df.apply(cal_price_prob,args=[i,u,d,s,'price'],axis=1)
    price_df['Prob'] = price_df.apply(cal_price_prob,args=[i,u,d,s,'prob'],axis=1)
    price_df['Value']= s
    if t=='UAO':
        price_df_temp = price_df[[f'P({i})' for i in range(n+1)]]
        for i in range(len(price_df.index)):
            ma = max(price_df_temp.loc[i].values)
            if ma > l:
                price_df['Value'].loc[i] = 0
            else:
                if price_df[f'P({n})'].loc[i] <= x:
                    price_df['Value'].loc[i] = 0
                else:
                    price_df['Value'].loc[i] = price_df[f'P({n})'].loc[i] - x
        price_df['Temp'] =price_df['Prob'] *price_df['Value']
        sum_UAO = sum(price_df["Temp"].values)
        price_df = price_df.drop(['Temp'],axis=1)
        dic['UAO'] = sum_UAO
    elif t=='UAI':
        price_df_temp = price_df[[f'P({i})' for i in range(n+1)]]
        for i in range(len(price_df.index)):
            ma = max(price_df_temp.loc[i].values)
            if ma < l:
                price_df['Value'].loc[i] = 0
            else:
                if price_df[f'P({n})'].loc[i] <= x:
                    price_df['Value'].loc[i] = 0
                else:
                    price_df['Value'].loc[i] = price_df[f'P({n})'].loc[i] - x
        price_df['Temp'] =price_df['Prob'] *price_df['Value']
        sum_UAI = sum(price_df["Temp"].values)
        price_df = price_df.drop(['Temp'],axis=1)
        dic['UAI'] = sum_UAI
    elif t=='normal':
        def cal_normal(a1):
            val = a1[f'P({n})']
            if val <= x:
                return 0
            else:
                return val - x 
        price_df['Value'] = price_df.apply(cal_normal,axis=1)
        price_df['Temp'] =price_df['Prob'] *price_df['Value']
        sum_normal = sum(price_df["Temp"].values)
        price_df = price_df.drop(['Temp'],axis=1)
        dic['normal'] = sum_normal

    print(f'The value of {t} European call option is: {dic[t]:.3f}$')
    print(f'The data table for {t} European call option is below:')
    return price_df

#### a. Find the value of this option.

#### ANSWER

In [5]:
cal_exotic_option(
    S2,X2,L,u,d,5,'UAO',0
)

The value of UAO European call option is: 3.164$
The data table for UAO European call option is below:


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,w,P(0),P(1),P(2),P(3),P(4),P(5),Prob,Value
0,uuuuu,100.0,112.000000,125.440000,140.492800,157.351936,176.234168,0.023352,0.0
1,uuuud,100.0,112.000000,125.440000,140.492800,157.351936,140.492800,0.026154,0.0
2,uuudu,100.0,112.000000,125.440000,140.492800,125.440000,140.492800,0.026154,0.0
3,uuudd,100.0,112.000000,125.440000,140.492800,125.440000,112.000000,0.029293,0.0
4,uuduu,100.0,112.000000,125.440000,112.000000,125.440000,140.492800,0.026154,0.0
5,uudud,100.0,112.000000,125.440000,112.000000,125.440000,112.000000,0.029293,12.0
6,uuddu,100.0,112.000000,125.440000,112.000000,100.000000,112.000000,0.029293,12.0
7,uuddd,100.0,112.000000,125.440000,112.000000,100.000000,89.285714,0.032808,0.0
8,uduuu,100.0,112.000000,100.000000,112.000000,125.440000,140.492800,0.026154,0.0
9,uduud,100.0,112.000000,100.000000,112.000000,125.440000,112.000000,0.029293,12.0


#### b. Which option is more expensive: the European call, or the UAO European call?<br>

#### ANSWER

The price of the normal option is more expensive than the UAO European call option 

$$(19.202 > 1.655)$$

In [6]:
cal_exotic_option(
    S2,X2,L,u,d,5,'normal',0
)

The value of normal European call option is: 10.591$
The data table for normal European call option is below:


,w,P(0),P(1),P(2),P(3),P(4),P(5),Prob,Value
0,uuuuu,100.0,112.000000,125.440000,140.492800,157.351936,176.234168,0.023352,76.234168
1,uuuud,100.0,112.000000,125.440000,140.492800,157.351936,140.492800,0.026154,40.492800
2,uuudu,100.0,112.000000,125.440000,140.492800,125.440000,140.492800,0.026154,40.492800
3,uuudd,100.0,112.000000,125.440000,140.492800,125.440000,112.000000,0.029293,12.000000
4,uuduu,100.0,112.000000,125.440000,112.000000,125.440000,140.492800,0.026154,40.492800
5,uudud,100.0,112.000000,125.440000,112.000000,125.440000,112.000000,0.029293,12.000000
6,uuddu,100.0,112.000000,125.440000,112.000000,100.000000,112.000000,0.029293,12.000000
7,uuddd,100.0,112.000000,125.440000,112.000000,100.000000,89.285714,0.032808,0.000000
8,uduuu,100.0,112.000000,100.000000,112.000000,125.440000,140.492800,0.026154,40.492800
9,uduud,100.0,112.000000,100.000000,112.000000,125.440000,112.000000,0.029293,12.000000


#### c. What is the advantage of the up-and-out European call option?

The advantage of the UAO call option is that, if we could estimate the variation range of a stock price with a good confidence level (95%), we could have the same payoff as normal European call option - despsite the UAO option being much cheaper than the normal European option.

#### d. There is another exotic option called an Up-and-In (UAI) European Call option... 

Without running a binomial tree, what is the price of UAI with the same strike and maturity as the European call, and the same strike, maturity, and barrier as the UAO. Assume the current stock price is below the barrier. (Hint: Think of a parity using the European call, UAI, and UAO).

#### ANSWER:

We have the following table of the prices of the options:

|Option|UAO|UAI|Normal|
|--|--|--|--|
|Price|3.155 |16.047 |19.202|

Thus, we can conclude that: 

$$Pr(UAO) + Pr(UAI) = Pr(Normal)$$

In [7]:
cal_exotic_option(
    S2,X2,L,u,d,5,'UAI',0
)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


The value of UAI European call option is: 7.427$
The data table for UAI European call option is below:


,w,P(0),P(1),P(2),P(3),P(4),P(5),Prob,Value
0,uuuuu,100.0,112.000000,125.440000,140.492800,157.351936,176.234168,0.023352,76.234168
1,uuuud,100.0,112.000000,125.440000,140.492800,157.351936,140.492800,0.026154,40.492800
2,uuudu,100.0,112.000000,125.440000,140.492800,125.440000,140.492800,0.026154,40.492800
3,uuudd,100.0,112.000000,125.440000,140.492800,125.440000,112.000000,0.029293,12.000000
4,uuduu,100.0,112.000000,125.440000,112.000000,125.440000,140.492800,0.026154,40.492800
5,uudud,100.0,112.000000,125.440000,112.000000,125.440000,112.000000,0.029293,0.000000
6,uuddu,100.0,112.000000,125.440000,112.000000,100.000000,112.000000,0.029293,0.000000
7,uuddd,100.0,112.000000,125.440000,112.000000,100.000000,89.285714,0.032808,0.000000
8,uduuu,100.0,112.000000,100.000000,112.000000,125.440000,140.492800,0.026154,40.492800
9,uduud,100.0,112.000000,100.000000,112.000000,125.440000,112.000000,0.029293,0.000000
